In [0]:
if spark.conf.get("spark.databricks.clusterUsageTags.clusterUnityCatalogMode") != "USER_ISOLATION":
    print("Dette er ikke et felles cluster!")
else:
    print("Kjøres på korrekt jobbkluster med single-user mode.")

In [0]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121


In [0]:
!nvidia-smi

In [0]:
import torch
print(torch.version.cuda)
print(torch.backends.cuda.is_built())
print(torch.cuda.is_available())

In [0]:
#%run "./dataProcessing/transform"

In [0]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
#from torch.utils.tensorboard import SummaryWriter
import sys
import os
import mlflow
import mlflow.pytorch

from dataProcessing.dataset import SnuplassDataset, load_numpy_split_stack
from model.unet import UNet
from dataProcessing.augmentation_config import augmentation_profiles
#from utils import iou_pytorch, acc_pytorch

In [0]:
def main():
    mlflow.pytorch.autolog()  # Lagrer modellen under Experiments. Kan hente modellen med model = mlflow.pytorch.load_model("runs://model")

    cfg = augmentation_profiles["default"]
    batch_size = 8
    num_epochs = 1
    learning_rate = 1e-3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #log_dir = "runs/snuplasser"
    #writer = SummaryWriter(log_dir=log_dir)

    train_ids, val_ids, _ = load_numpy_split_stack(
        image_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/img/",
        mask_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/lab/",
        dom_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/dom/",
    )

    train_dataset = SnuplassDataset(
        image_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/img/",
        mask_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/lab/",
        dom_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/dom/",
        file_list=train_ids,
        transform=None,  # ratio=None for baseline
        # For å bruke augmentering, sett ratio til en verdi mellom 0 og 1
    )

    val_dataset = SnuplassDataset(
        image_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/img/",
        mask_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/lab/",
        dom_dir="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/dom/",
        file_list=val_ids,
        transform=None,
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = UNet(n_channels=4, n_classes=1, bilinear=False).to(
        device
    )  # bare å bytte modell

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    with mlflow.start_run(run_name="UNet_baseline_4ch"):
        for epoch in range(num_epochs):
            # Trening
            model.train()
            total_loss = 0

            for images, masks in tqdm(
                train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"
            ):
                images, masks = images.to(device).float(), masks.to(device).float()
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs.squeeze(1), masks)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_train_loss = total_loss / len(train_loader)
            print(f"\nTrain loss: {avg_train_loss:.4f}")

            #writer.add_scalar("Tap/Trening", avg_train_loss, epoch)

            # Validering
            model.eval()
            val_loss = 0.0
            #val_ious = []
            #val_accs = []
            with torch.no_grad():
                for images, masks in tqdm(
                    val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"
                ):
                    images, masks = images.to(device).float(), masks.to(device).float()
                    outputs = model(images)
                    loss = criterion(outputs.squeeze(1), masks)
                    val_loss += loss.item()

                    # Beregn IoU og accuracy
                    #predictions = (
                    #    torch.sigmoid(outputs) > 0.5
                    #).int()  # Konverterer til binære prediksjoner
                    #iou = iou_pytorch(predictions, masks.int())
                    #acc = acc_pytorch(predictions, masks.int())
                    #val_ious.append(iou.item())
                    #val_accs.append(acc.item())

            avg_val_loss = val_loss / len(val_loader)
            print(f"Val loss: {avg_val_loss:.4f}")
            #avg_iou = sum(val_ious) / len(val_ious)
            #avg_acc = sum(val_accs) / len(val_accs)

            #writer.add_scalar("Tap/Validering", avg_val_loss, epoch)
            #writer.add_scalar("Metrikker/IoU", avg_iou, epoch)
            #writer.add_scalar("Metrikker/Accuracy", avg_acc, epoch)

        #writer.close()

    print("✅ Trening ferdig")

main()
     